In [7]:
import subprocess as sp
import re
import os
from glob import glob

In [8]:
c2nim = "../c2nim/c2nim"
default_lib = "av.c2nim"

inc_sys = re.compile(r"(?<=\<).+?(?=\>)")
inc_loc = re.compile(r"\"(.+?)\"")

In [9]:
discard_mods = ["avconfig", "common", "bprint", "avstring", "intreadwrite", "mem", "error",
               "log", "mathematics", "rational", "version"]
new_incs = ["../nim_ffmpeg_common.h"]

ffmpeg_libs = set(["libavutil", "libavcodec", "libavformat", "libswscale"])

In [10]:
def dynlib_header(lib):
    
    l = lib.split("lib",1)[-1]
    
    h = """\
    #ifdef C2NIM
    #  dynlib {l}dll
    #  cdecl
    #  if defined(windows)
    #    define {l}dll "{l}.dll"
    #  elif defined(macosx)
    #    define {l}dll "{lib}.dylib"
    #  else
    #    define {l}dll "{lib}.so"
    #  endif
    #endif
    """.format(l=l, lib=lib)
    
    return h

In [11]:
def purge_includes(fn, matchers): 

    lines = []
    includes = [[] for _ in matchers]
    
    with open(fn, "r") as f:
        
        for i, l in enumerate(f):
            if "#include" in l:
                lines.append(f"//{l.strip()} // #{i}\n")
                
                total_inc = 0
                for inc, m in zip(includes, matchers):
                    
                    what = m.findall(l)
                    
                    nr_incs = len(what)
                    assert nr_incs <= 1
                    
                    total_inc += nr_incs 
                    inc += what
                    
                assert total_inc >= 1 
                    
            else:
                lines.append(l)

        return "".join(lines), includes

In [12]:
def preproc(string):
    
    args = ['cpp','-E', '-P']
    cpp = sp.Popen(args, universal_newlines=True, stdin=sp.PIPE, stderr=sp.PIPE, stdout=sp.PIPE)
    out, err = cpp.communicate(string)
    ret = cpp.poll()
    
    if ret != 0:
        raise RuntimeError(f"Pre-processing failed because of {err} with exit code:", ret)
    
    return out, err

In [13]:
def conv2nim(fn_in, folder_out):
    
    os.makedirs(folder_out, exist_ok=True)
    fn_out = os.path.join(folder_out, fn_in.rsplit(os.sep,1)[-1].rsplit(".h", 1)[0]+'.nim')
    
    args = [c2nim, default_lib, fn_in, f'--out:{fn_out}']
    #args = [c2nim, default_lib, fn_in]
    
    print(args)
    
    c2n = sp.Popen(args, universal_newlines=True, stdin=sp.PIPE, stderr=sp.PIPE, stdout=sp.PIPE)
    out, err = c2n.communicate()
    ret = c2n.poll()
    
    if ret != 0:
        raise RuntimeError(f"c2nim conversion failed because of {err} with exit code:", ret)
    
    return out, err

In [14]:
def preproc_lib(lib, pp_lib, exclude_mods=[]):
    
    sys_incs = {}
    loc_incs = {}

    try:
        os.mkdir(pp_lib)
    except FileExistsError:
        pass

    
    header = dynlib_header(lib)

    for fn in glob(os.path.join(lib, '*.h')):

        mod = fn.replace(lib+os.sep, "")
        if all(m not in mod for m in exclude_mods):

            pp, (si, li) = purge_includes(fn, (inc_sys, inc_loc))
            sys_incs[mod] = si
            #loc_incs[mod] = li

            #Change to local import
            li_modified = []
            other_libs = (ffmpeg_libs-set(lib))
            for inc in li:
                
                if all(m not in inc for m in exclude_mods):
                
                    if inc.startswith(lib):
                        li_modified.append(inc.replace(lib+os.sep, ""))
                    elif any(inc.startswith(l) for l in other_libs):
                        li_modified.append('..'+os.sep+inc)
                    else:
                        li_modified.append(inc)

            loc_incs[mod] = li_modified

            print(mod, li_modified)
            
            fn_pp = os.path.join(pp_lib, mod)

            with open(fn_pp, "w") as f:
                pp_flat, _ = preproc(pp)
                
                
                
                #Add back includes such that they stay local
                incs = "\n".join(f"#include \"{i}\"" for i in (new_incs + li_modified)) + "\n"
                
                
                f.write(header + incs + pp_flat)
        
    return sys_incs, loc_incs

In [15]:
def c2nim_lib(lib, pp_lib):
    
    for fn in glob(os.path.join(pp_lib, '*.h')):

        try:
            out, err = conv2nim(fn, lib)
        except RuntimeError as e:
            print(fn, e)

In [16]:
for lib in ffmpeg_libs:
    pp_lib = "pp-"+lib
    preproc_lib(lib, pp_lib, discard_mods)

avcodec.h ['../libavutil/samplefmt.h', '../libavutil/attributes.h', '../libavutil/avutil.h', '../libavutil/buffer.h', '../libavutil/cpu.h', '../libavutil/channel_layout.h', '../libavutil/dict.h', '../libavutil/frame.h', '../libavutil/pixfmt.h']
avdct.h ['../libavutil/opt.h']
avfft.h []
d3d11va.h []
dirac.h ['avcodec.h']
dv_profile.h ['../libavutil/pixfmt.h', 'avcodec.h']
dxva2.h []
jni.h []
mediacodec.h ['avcodec.h']
qsv.h ['../libavutil/buffer.h']
vaapi.h ['../libavutil/attributes.h']
vda.h ['avcodec.h']
vdpau.h ['../libavutil/attributes.h', 'avcodec.h']
videotoolbox.h ['avcodec.h']
vorbis_parser.h []
xvmc.h ['../libavutil/attributes.h', 'avcodec.h']
avformat.h ['../libavcodec/avcodec.h', '../libavutil/dict.h', 'avio.h']
avio.h ['../libavutil/dict.h']
swscale.h ['../libavutil/avutil.h', '../libavutil/pixfmt.h']
adler32.h ['attributes.h']
aes.h ['attributes.h']
aes_ctr.h ['attributes.h']
attributes.h []
audio_fifo.h ['avutil.h', 'fifo.h', 'samplefmt.h']
avassert.h ['avutil.h']
avutil.h

In [17]:
for lib in ffmpeg_libs:
    for fn in glob(os.path.join(lib, '*.nim')):
        os.remove(fn)

In [18]:
for lib in ffmpeg_libs:
    pp_lib = "pp-"+lib
    c2nim_lib(lib, pp_lib)

['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/avcodec.h', '--out:libavcodec/avcodec.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/avdct.h', '--out:libavcodec/avdct.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/avfft.h', '--out:libavcodec/avfft.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/d3d11va.h', '--out:libavcodec/d3d11va.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/dirac.h', '--out:libavcodec/dirac.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/dv_profile.h', '--out:libavcodec/dv_profile.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/dxva2.h', '--out:libavcodec/dxva2.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/jni.h', '--out:libavcodec/jni.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/mediacodec.h', '--out:libavcodec/mediacodec.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/qsv.h', '--out:libavcodec/qsv.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp-libavcodec/vaapi.h', '--out:libavcodec/vaapi.nim']
['../c2nim/c2nim', 'av.c2nim', 'pp